This Jupyter notebook contains code for the **datasong** project. 

## Set things up

In [1]:
## Import all the things

import pyglet
import pandas as pd
from tabulate import tabulate

## Code to play sound

Popup window will open when code is executed. When you close window, sound will stop.

In [2]:
def launch_window(data, clock, met):
    """ Launches window and displays content. 
    
    Args:
        data: Pandas DataFrame
        clock: Pyglet clock object
        met: Metronome object        
    """
    
    window = pyglet.window.Window(900, 500, caption='datasong Sound Player')
    
    # Display top label
    
    row_col_count = 'DataFrame: {row} x {column}'.format(row=data.shape[0], column=data.shape[1])
    
    label1 = pyglet.text.Label('Don\'t Panic', font_name='Whitney', font_size=18,
                              x=window.width//2, y=window.height//2,
                              anchor_x='center', anchor_y='bottom')
    
    label2 = pyglet.text.Label(row_col_count, font_name='Whitney', font_size=13,
                              x=window.width//2, y=window.height//1.1,
                              anchor_x='center', anchor_y='bottom')
    
    @window.event
    def on_draw():
        window.clear()
        label1.draw()
    
    # Display streaming rows of data
    
    def update_screen(dt, data, met):
        """ Update window content with streaming data. """
        
        current_index = met.next_beat()
        
        if current_index <= 14:
            index_list = list(range(0,current_index+1))

        if current_index > 14:
            index_list = list(range(current_index-14,current_index+1))
        
        rows_to_tabulate = [data.iloc[i].tolist() for i in index_list]      
    
        data_text = tabulate(rows_to_tabulate, headers = dat.columns.tolist())

        data_label = pyglet.text.Label(data_text,
                              font_name='Ubuntu Mono',
                              font_size=10, width=700, x=window.width//2, y=window.height//1.5,
                              anchor_x='center', anchor_y='center', multiline=True)

        @window.event
        def on_draw():
            window.clear()
            label2.draw()
            data_label.draw()
    
    clock.schedule_interval(update_screen, 2, data, met)
        
    @window.event
    def on_close():
        clock.unschedule(update)

In [3]:
def play_background(audio_file):
    """ Play the background song. """
    
    # Instantiate player; remove any preexisting audio from queue

    global bplayer
    
    try:
        bplayer
    except NameError: bplayer = None

    if bplayer is None:
        bplayer = pyglet.media.Player()
    else:
        bplayer.delete()
    
    # Add new audio file to queue

    bsource = pyglet.media.load(audio_file, streaming=False)

    bplayer.queue(bsource) 
    bplayer.play()

In [4]:
def play_ding(audio_file):
    """ Play DING. """

    # Instantiate player; remove any preexisting audio from queue
    
    global dplayer
    
    try:
        dplayer
    except NameError: dplayer = None

    if dplayer is None:
        dplayer = pyglet.media.Player()
    else:
        dplayer.delete()
        
    # Add new audio file to queue
        
    dsource = pyglet.media.load(audio_file, streaming=False)
    dplayer.queue(dsource)
    dplayer.play()

In [5]:
class Metronome:
    """ Class that remembers how many beats have elapsed since object instantiation.
    
    Attributes:
        beats_elapsed (int): Number of beats elapsed.
    """
    
    def __init__(self):
        self.beats_elapsed = -1
        
    def next_beat(self):
        self.beats_elapsed += 1
        return self.beats_elapsed

In [6]:
def update(dt, data, col, met):
    """ Runs at regular intervals within event loop to produce DING. 
    
    Args:
        dt: This needs to be here for clock.schedule_update() to work
        data: Pandas DataFrame
        col: Column to play DINGS
        met: Metronome object
    """
    
    current_index = met.next_beat()
    
    if data[col].iloc[current_index] == 1:
        play_ding('sound-files/DING_StableB_01_1_NZD.wav')

In [7]:
class Conductor:
    """
    Class that ingests and sonifies data.
    """

    def __init__(self, data):
        self.data = data
        
        if 'DataFrame' not in str(type(self.data)): 
            raise Exception('Not a pandas DataFrame.')
    
    def background(self, background):
        """ Select genre of music to use for sonification. """
        
        self.background = background
    
    def ding(self, col):
        """ Assign columns for DINGS. """
        
        self.ding_col = col
    
        
    def play(self):
        """ Sonify and play data. """
        
        # Print text to console
        
        print('Sonifying data...')
        print('Close sound player to stop playback and exit.')
        
        # Play background
        
        play_background(self.background)
        
        # Loop through DataFrame - play dings     
        
        metronome_ding = Metronome()
        clock = pyglet.clock
        clock.schedule_interval(update, 2, self.data, self.ding_col, metronome_ding)
        
        # Launch window and play sound
        
        metronome_screen = Metronome()
        launch_window(self.data, clock, metronome_screen)
        
        pyglet.app.run()

## Testing Sandbox

Run code below to test.

In [ ]:
dat = pd.read_csv('data/intrusion_detection.csv')
dat = dat[['timestamp', 'counter(mins)', 'Dos', 'scan44', 'scan11', 'nerisbotnet', 'blacklist']]

test = Conductor(dat)
test.background('sound-files/Stable_A_16_NZD.wav')
test.ding('blacklist')
test.play()

Sonifying data...
Close sound player to stop playback and exit.
